In [ ]:
from tools.stat_tools.stat_tools import *
from tools.visualization.visualization import *
from tools.stat_tools.bt_utils import *

import pandas as pd
import random
import numpy as np
import datetime as dt
import math
import matplotlib.pyplot as plt
import plotly.express as px
import random
import os
import glob

pd.options.mode.chained_assignment = None 

## Lectura de Backtest

In [ ]:
magics = ['22014', '22070', '22093', '18075', '25184', '18062', '22007', '25144', '25354', '25024', '25098', '25232', '25305', 
          '25329', '25350', '25400']
bt_df = read_backtest(magics, verbose = True)
# bt_df

In [ ]:
# bt_df.to_pickle("./backtest_data.pkl")
# bt_df = pd.read_pickle("./backtest_data.pkl")

## Correlacion de Sets

In [ ]:
df_series = bt_df[['Close_Date', 'Magic_Number', 'Profit']]
# df_series = df_series.drop(df_series[df_series['Magic_Number'].isin(dqg_new)].index.tolist())

df_series['Close_Date'] = df_series['Close_Date'].dt.strftime('%Y-%m-%d')
df_series = df_series.groupby(['Close_Date', 'Magic_Number'], as_index = False)['Profit'].sum().reset_index()

df_pivot = df_series.pivot(index = ['index', 'Close_Date'], columns = 'Magic_Number', values = 'Profit').fillna(0).reset_index()
df_pivot =  df_pivot.drop(['index'], axis=1)
corr = df_pivot.groupby('Close_Date').sum().corr().abs()

corr.style.background_gradient(cmap='coolwarm')

<h1><center>DQG</center></h1>

## Creacion de Portafolio

In [ ]:
df_backtest = bt_df.copy()
# Parametros de entrada
max_sets = 30
weights_dict = {'Exp_Profit': 1, 'Profit_per_Day': 1, 'PF_nor': 1, 'R2_KPI': 1, 'Win/DD_Ratio_Month': 1, 
                'Max_Profit': 1, 'Max_DD': 2, 'DD_Duration': 1, 'Bkn_KPI': 1, 'Sharpe_Ratio': 1, 'SQN': 1,
                'Win_divergence': 0}
treshold = 0.25

# Cleaning df_backtest
# df_backtest = df_backtest.drop(df_backtest[df_backtest['Magic_Number'].isin(qdq)].index.tolist())

magics_available = df_backtest.Magic_Number.unique()

corr = corr[corr.index.isin(magics_available)]
corr = corr[[*magics_available]]

# Cesta Nueva
# qdq = ['22014', '22070', '22093', '18075', '25184', '18062', '25144', '25354']

dqg = ['22018', '22058', '22068', '18058', '18065', '18024', '19106', '20025', '22015', '22020', '18027', '25015', '22000', 
       '18076', '22172', '25006', '25035', '25099', '18090', '22021', '22035', '22059', '22087', '22129', '25013', '25090', 
       '25355']

In [ ]:
dqg_new, df_ins = portfolio_creation(df_backtest, dqg, weights_dict, corr, treshold = treshold)

In [ ]:
magic_list = dqg_new
color_list = sets_color(magic_list)
print("----- Total sets: {}".format(len(magic_list)))
print("\n---- Sets: {}".format(magic_list))

df_magic_evolution = df_ins.set_magic_evolution()
df_port_evolution = df_ins.set_port_evolution(df_backtest[df_backtest.Magic_Number.isin(magic_list)])
df_resumen = df_ins.profit_per_period() 

df_port = df_ins.get_stat('DQG', df_backtest[df_backtest.Magic_Number.isin(magic_list)])
display(df_port)


fig_ins = sets_visualization(df_backtest, magic_list)
fig = fig_ins.evolution_plot(df_port_evolution, df_magic_evolution, df_resumen, color_list, plot_pf = False, )

fig.show()

## Superposicion de DD

In [ ]:
fig = fig_ins.plot_dd(color_list)
fig.show()

## Regresion

In [ ]:
fig = fig_ins.plot_regression(color_list)
fig.show()

<h1><center>QDQ</center></h1>

## Creacion de Portafolio

In [ ]:
# Cleaning df_backtest
dqg_new= dqg
df_backtest = bt_df.copy()
df_backtest.rename(columns = {'Tiempo': 'Close_Date', 'Beneficios': 'Profit', 
                              'Volumen': 'Units/Lots'}, inplace= True)
df_backtest['Open_Date'] = df_backtest.Close_Date
df_backtest['Action'] = 'Buy'
df_backtest['Pips'] = 100
df_backtest = df_backtest.drop(df_backtest[df_backtest['Magic_Number'].isin(dqg_new)].index.tolist())

magics_available = df_backtest.Magic_Number.unique()
corr = corr[corr.index.isin(magics_available)]
corr = corr[[*magics_available]]



qdq = ['22014', '22070', '22093', '18075', '25184', '18062', '25144', '25354']
qdq, df_ins = portfolio_creation(df_backtest, qdq, weights_dict, corr, treshold = treshold)

In [ ]:
magic_list = qdq
color_list = sets_color(magic_list)
print("----- Total sets: {}".format(len(magic_list)))
print("\n---- Sets: {}".format(magic_list))

df_magic_evolution = df_ins.set_magic_evolution()
df_port_evolution = df_ins.set_port_evolution(df_backtest[df_backtest.Magic_Number.isin(magic_list)])
df_resumen = df_ins.profit_per_period() 

df_port = df_ins.get_stat('DQG', df_backtest[df_backtest.Magic_Number.isin(magic_list)])
display(df_port)

fig_ins = sets_visualization(df_backtest, magic_list)
fig = fig_ins.evolution_plot(df_port_evolution, df_magic_evolution, df_resumen, color_list, plot_pf = False)

fig.show()

## Superposicion de DD

In [ ]:
fig = fig_ins.plot_dd(color_list)
fig.show()

## Regresion

In [ ]:
fig = fig_ins.plot_regression(color_list)
fig.show()

## Portfolio Correlation

In [ ]:
df_series = bt_df[['Tiempo', 'Magic_Number', 'Beneficios']]
total_sets = dqg_new + qdq
df_series = df_series[df_series.Magic_Number.isin(total_sets)]
df_series['Portfolio'] = None
df_series.loc[df_series.Magic_Number.isin(dqg_new), 'Portfolio'] = 'DQG'
df_series.loc[df_series.Magic_Number.isin(qdq), 'Portfolio'] = 'NP'
df_series = df_series[['Tiempo', 'Portfolio', 'Beneficios']]

df_series['Tiempo'] = df_series['Tiempo'].dt.strftime('%Y-%m-%d')
df_series = df_series.groupby(['Tiempo', 'Portfolio'], as_index = False)['Beneficios'].sum().reset_index()

df_pivot = df_series.pivot(index = ['index', 'Tiempo'], columns = 'Portfolio', values = 'Beneficios').fillna(0).reset_index()
df_pivot =  df_pivot.drop(['index'], axis=1)
corr = df_pivot.groupby('Tiempo').sum().corr().abs()

corr.style.background_gradient(cmap='coolwarm')